# pdf2md

In [1]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

/home/zyx/anaconda3/envs/paperins/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
converter = PdfConverter(
    artifact_dict=create_model_dict(),
)

Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


In [47]:
rendered = converter("./data/1.pdf")
text, _, images = text_from_rendered(rendered)

Running OCR Error Detection: 100%|██████████| 7/7 [00:00<00:00, 230.24it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Texify inference: 100%|██████████| 1/1 [00:01<00:00,  1.86s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  2.16it/s]


In [ ]:
with open("./data/output.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [5]:
with open("./data/output.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [6]:
import re

def clean_markdown(text):
    # 保留换行符拆行
    lines = text.splitlines(keepends=True)

    # 删除 Markdown 和 HTML 图片
    image_pattern = re.compile(r"!\[[^\]]*\]\([^)]*\)|<img[^>]*>", re.IGNORECASE)
    lines = [image_pattern.sub('', line) for line in lines]

    # 找到第一个标题中包含 appendix/references 的行，截断其之前内容
    cutoff_index = len(lines)
    for i, line in enumerate(lines):
        if '#' in line and any(keyword in line.lower() for keyword in ['appendix', 'references']):
            cutoff_index = i
            break

    cleaned_lines = lines[:cutoff_index]

    return ''.join(cleaned_lines)  # 不做换行或压缩，保持原始结构

In [7]:
texts = text
text1 = clean_markdown(texts)

# 生成问题

In [9]:
from openai import OpenAI

def gpt_chat(sys: str, user: str, provider: str) -> str:
    config = {
        "doubao": {
            "base_url": "https://ark.cn-beijing.volces.com/api/v3",
            "api_key": "d051e32c-ec24-445e-a7cc-4b7bc5a54899",
            "model": "doubao-1-5-pro-32k-250115"
        },
        "kimi": {
            "base_url": "https://api.moonshot.cn/v1",
            "api_key": "sk-zhMCEZyyOFcNm3OAKmWHDCnJVIABCbHIEemu8pTTjWDg7n1o",
            "model": "moonshot-v1-32k"
        },
        "qwen72": {
            "base_url": "https://dashscope.aliyuncs.com/compatible-mode/v1",
            "api_key": "sk-3b32fb0739b2411bb9c098e6f9b56a3e",
            "model": "qwen2.5-72b-instruct"
        },
        "qwen32": {
            "base_url": "https://dashscope.aliyuncs.com/compatible-mode/v1",
            "api_key": "sk-3b32fb0739b2411bb9c098e6f9b56a3e",
            "model": "qwen2.5-32b-instruct"
        },
        "deepseek": {
            "base_url": "https://api.deepseek.com/",
            "api_key": "sk-3f4a482416744badbd6ee1ff7d268026",
            "model": "deepseek-chat"
        }
    }

    if provider not in config:
        raise ValueError(f"Unknown provider: {provider}")

    cfg = config[provider]
    client = OpenAI(
        base_url=cfg["base_url"],
        api_key=cfg["api_key"]
    )

    completion = client.chat.completions.create(
        model=cfg["model"],
        messages=[
            {"role": "system", "content": sys},
            {"role": "user", "content": user}
        ]
    )
    return completion.choices[0].message.content


In [10]:
GET_QUESTIONS_TEMPLATE = """
Hello, I need your expertise to generate a set of thought-provoking questions based on the provided context.

<context>{context}</context>

Instructions:
* Generate a list of questions.
* Ensure that each question is realistically derived from the given context.
* Each question should be complex, demanding careful consideration and multiple steps to answer.
* Enclose each question within <question></question> tags.
* Do not number the questions.
* Generate up to {max_questions} questions.
"""

GET_QUESTIONS_SYSTEM_MESSAGE = """
You are a distinguished professor with extensive expertise in analyzing academic papers and formulating insightful questions. 
Your profound knowledge of the subject allows you to identify intricate questions that demand thorough contemplation. 
Your primary objective is to assist the user in generating questions that are intellectually challenging and necessitate deep thought to resolve.
"""

In [ ]:
max_questions = 3
context = GET_QUESTIONS_TEMPLATE.format(
    context=text1,
    max_questions=max_questions
)
questions = gpt_chat(
    sys=GET_QUESTIONS_SYSTEM_MESSAGE,
    user=context,
    provider="deepseek"
)


In [12]:
def extract_html_tags(text, keys):
    content_dict = {}
    keys = set(keys)
    for key in keys:
        pattern = f"<{key}>(.*?)</{key}>"
        matches = re.findall(pattern, text, re.DOTALL)
        # print(matches)
        if matches:
            content_dict[key] = [match.strip() for match in matches]
    return content_dict

def _validate_tasks(out):
    questions = extract_html_tags(out, ["question"])
    if "question" not in questions:
        return (
            out,
            False,
            f"Error: you did not generate questions within the <question></question> tags",
        )
    questions = questions["question"]
    # Check that there are at most max_questions questions
    if len(questions) > 3:
        return (
            out,
            False,
            f"Error: you can only ask at most {3} questions, but you asked {len(questions)}.",
        )

    return (questions, out), True, ""

In [18]:
a= _validate_tasks(questions)
a[0][0]

['How does the assimilation of MODIS AOD and MOPITT CO retrievals in the CMAQ model impact the accuracy and uncertainty of long-term air quality reanalysis, particularly in regions with sparse ground-based monitoring networks, and what are the implications for air quality management policies?',
 'What are the underlying factors contributing to the observed discrepancies between modeled and observed PM2.5 concentrations, especially in regions like the Pacific Northwest where wildfire activity has increased, and how might these discrepancies influence the reliability of trend analyses in such areas?',
 'Given the varying performance of the WRF-CMAQ system across different meteorological parameters (e.g., temperature vs. wind speed) and pollutant types (e.g., ozone vs. PM2.5), what methodological improvements could be implemented to reduce biases and enhance the utility of the reanalysis dataset for stakeholders?']

# 回答问题

In [32]:
GET_ANSWER_TEMPLATE = """
Hello, I need your expertise to provide detailed answers to the questions based on the provided context.

<questions>{questions}</questions>

<context>{context}</context>

Instructions:
* Provide detailed answer to the question.
* Ensure the answers are comprehensive and cover all aspects of the questions.
* Use the context provided to support your answers.
* Use clear connectors such as "First", "Next," "Then," and "Finally" to link each step.  
* Enclose each answer within <answer></answer> tags.
* Do not include content unrelated to answering the questions.
"""

GET_ANSWER_SYSTEM_MESSAGE = """
You are an expert in providing logical, multi-step reasoning and detailed answers to questions based on given contexts. 
Your role is to analyze the questions and the provided context thoroughly, and to generate answers that are not only accurate but also rich in detail. 
You should ensure that your answers address all parts of the questions and are supported by the context.
Your goal is to deliver answers that are logical, well-structured, and easy to follow.
"""

In [33]:
qs_context = GET_ANSWER_TEMPLATE.format(
    questions=a[0][0][1],
    context=text1
)
answer = gpt_chat(sys=GET_ANSWER_SYSTEM_MESSAGE, user=qs_context, provider="deepseek")

In [34]:
final = extract_html_tags(answer, ["answer"])

In [35]:
print(final["answer"][0])

The observed discrepancies between modeled and observed PM2.5 concentrations, particularly in regions like the Pacific Northwest where wildfire activity has increased, can be attributed to several underlying factors:

1. **Uncertainties in Emission Inventories**: 
   - The study highlights that anthropogenic and biomass burning emission estimates vary significantly across inventories (by factors of 1.16–2.94 for anthropogenic emissions and 3.13–8.0 for fire emissions). In the Pacific Northwest, wildfires contribute substantially to PM2.5 levels, and uncertainties in fire emission inventories (e.g., FINN) can lead to discrepancies. For example, the model may underestimate the intensity or spatial distribution of wildfire emissions, which are critical for accurate PM2.5 simulations.

2. **Limitations in Data Assimilation**: 
   - The assimilation of MODIS AOD retrievals, while improving PM2.5 correlations, has limitations. The observation error for MODIS AOD increases with higher AOD val

In [31]:
a[0][0][1]

'What are the underlying factors contributing to the observed discrepancies between modeled and observed PM2.5 concentrations, especially in regions like the Pacific Northwest where wildfire activity has increased, and how might these discrepancies influence the reliability of trend analyses in such areas?'

In [27]:
print(text1)

Earth Syst. Sci. Data, 17, 1807–1834, 2025 https://doi.org/10.5194/essd-17-1807-2025 © Author(s) 2025. This work is distributed under the Creative Commons Attribution 4.0 License.



# **A long-term high-resolution air quality reanalysis with a public-facing air quality dashboard over the Contiguous United States (CONUS)**

Rajesh Kumar<sup>1</sup> , Piyush Bhardwaj1,\*, Cenlin He<sup>1</sup> , Jennifer Boehnert<sup>1</sup> , Forrest Lacey<sup>1</sup> , Stefano Alessandrini<sup>1</sup> , Kevin Sampson<sup>1</sup> , Matthew Casali<sup>1</sup> , Scott Swerdlin<sup>1</sup> , Olga Wilhelmi<sup>1</sup> , Gabriele G. Pfister<sup>1</sup> , Benjamin Gaubert<sup>1</sup> , and Helen Worden<sup>1</sup>

<sup>1</sup>NSF National Center for Atmospheric Research, Boulder, CO, USA \*now at: Center for Study of Science, Technology and Policy (CSTEP), Bengaluru, India

**Correspondence:** Rajesh Kumar (rkumar@ucar.edu)

Received: 13 May 2024 – Discussion started: 6 June 2024 Revised: 5 November 2024 – 

# 评估三元对